In [153]:
import cv2
import numpy as np
import onnxruntime as ort

In [154]:
model_path = r".\model_quantized.onnx"

In [155]:
video_path=r'.\violentVideo\vid_1.mp4'

In [156]:
def img_preprocessing(img):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    for i in range(3):
        img[:, :, i] = (img[:, :, i] - mean[i]) / std[i]
    img = np.transpose(img, (2, 0, 1))
    return img

In [157]:
def predict_on_video(video_path,onnx_model_path,n_fps_pred=7,n_frames_pred=8,n_frames_split=1):
    #loader model
    
    session = ort.InferenceSession(onnx_model_path)
    input_name=session.get_inputs()[0].name
    output_name=session.get_outputs()[0].name

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    fps=cap.get(cv2.CAP_PROP_FPS)
    interval=int(fps/n_fps_pred)
    X=[]
    violent_frames=[]
    tmp_violent_frames=[]
    counts=0
    while True:
        counts+=1
        ret,frame=cap.read()
        if not ret:
            break
        if(counts%interval==0):
            counts=0
            processed_frame=img_preprocessing(frame)
            X.append(processed_frame)
            tmp_violent_frames.append(frame)
        input_data=np.array([X.copy()],dtype=np.float32)
        if(len(input_data[0])!=n_frames_pred):
            pass
        else:
            outputs = session.run([output_name], {input_name: input_data})
            outputs[0][0]=np.exp(outputs[0][0])/np.sum(np.exp(outputs[0][0]))
            y_preds=np.argmax(outputs[0],axis=1)[0]
            # print(outputs[0][0])
            if(y_preds==1) and (outputs[0][0][1]<0.86) and (outputs[0][0][1]>0.6):
                if(len(violent_frames)>=n_frames_pred):
                    violent_frames.extend(tmp_violent_frames[-(n_frames_pred-int(n_frames_pred/n_frames_split)+1):])
                else:
                    violent_frames.extend(tmp_violent_frames)
                print(y_preds)
            X=X[-int(n_frames_pred/n_frames_split)+1:]
            tmp_violent_frames=tmp_violent_frames[-int(n_frames_pred/n_frames_split)+1:]
        cv2.imshow('Video',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        if not ret:
                break
    cap.release()
    cv2.destroyAllWindows()

    return violent_frames
        

In [158]:
n_fps_pred=12
n_frames_pred=14
violentFrames=predict_on_video(video_path=video_path,onnx_model_path=model_path,n_fps_pred=n_fps_pred,n_frames_pred=n_frames_pred)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [159]:
np.array(violentFrames).shape

(32, 360, 640, 3)

In [160]:
for i in range(len(violentFrames)):
    # Show each frame
    # if(i>=12):
    try:

        cv2.imshow('Video Frame', violentFrames[i])
    except:
        print(violentFrames[i])
        break
    # else:
    #     cv2.imshow('Video Frame', violentFrames[i])
    
    # Wait for 25 ms between frames (40 FPS). Adjust as needed.
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break  # Press 'q' to exit early

cv2.destroyAllWindows()

# Save Video

In [161]:
video_path.split('\\')[-1][:-4]

'vid_1'

In [162]:
if len(violentFrames)>0:
    height, width, layers = violentFrames[0].shape
    name=video_path.split('\\')[-1][:-4]
    output_path = f'./Out_video/{name}.mp4'
    fps = n_fps_pred  # Số frame mỗi giây
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Định dạng mã hóa MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


    for frame in violentFrames:
        out.write(frame)

 
    out.release()
    print("Video đã được lưu thành công tại", output_path)

Video đã được lưu thành công tại ./Out_video/vid_1.mp4
